In [1]:
## Walking experiments with anymal

import os, sys
sys.path.append('../lib')

import crocoddyl
from crocoddyl.utils.quadruped import plotSolution
from caracal.gaits import QuadrupedalGaitGenerator
from caracal.utils import plotContactPhaseDiagram
from caracal.params import SolverType
import caracal
import example_robot_data
import pinocchio
import pinocchio as pin
import numpy as np
import sys

import time
import copy
import matplotlib.pyplot as plt
from IPython.display import clear_output
import hppfcl
from pb_utils.visualize import set_q, vis_traj, create_primitives
from sl1m_env import setup_pybullet, generate_stair_small, create_obstacle_from_scene, load_robot_pinocchio
from anymal_utils import Simulator, create_contact_sequence_slim_to_caracal, compute_foot_pose, create_gait_trot, check_traj
import pybullet as p
import pybullet_data
from functools import partial
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--use_obstacle', type=bool, default=False)
parser.add_argument('--use_noise',type=bool, default=False)
parser.add_argument('--env_name',type=str, default='full_stair_small')
parser.add_argument('--obs_weight',type=float, default=100)
parser.add_argument('--solver_verbose',type=bool, default=False)
parser.add_argument('--use_gui',type=bool, default=False)
parser.add_argument('--N_ss',type=int, default=30)
parser.add_argument('--N_ds',type=int, default=45)
parser.add_argument('--MPC_HORIZON',type=int, default=60)
parser.add_argument('--w_penetration',type=float, default=100)

# args = parser.parse_args()

pybullet build time: Mar 26 2022 03:00:52


_StoreAction(option_strings=['--w_penetration'], dest='w_penetration', nargs=None, const=None, default=100, type=<class 'float'>, choices=None, help=None, metavar=None)

In [2]:
args = lambda : None

In [3]:
args.use_obstacle = True
args.use_noise = False
args.env_name = 'full_stair_small'
args.obs_weight = 200
args.solver_verbose = False
args.use_gui = True
args.N_ss = 30
args.N_ds = 45
args.MPC_HORIZON = 60

In [4]:
args.env_name = 'more_closer_full_stair_small_0.15'

In [5]:
which_computer = 'idiap' #'idiap'
if which_computer == 'laptop':
    urdf_name = '/opt/openrobots/share/example-robot-data/robots/anymal_b_simple_description/robots/anymal.urdf'
    mpc_filename = '/home/teguh/git/memmo/memmo_talos/anymal/generate_data_trot/data/config/mpc_params.yaml'
    DATA_ROOT = '/media/teguh/Data2/anymal_data_new/'
else:
    urdf_name = '/idiap/temp/tlembono/miniconda3/envs/robotpkg/share/example-robot-data/robots/anymal_b_simple_description/robots/anymal.urdf'
    mpc_filename = '/idiap/temp/tlembono/git/memmo_talos/anymal/generate_data_trot/data/config/mpc_params.yaml'
    DATA_ROOT = '/idiap/temp/tlembono/data_new/'

noise_t_set = np.load('data/noise_t_set.npy')
noise_y_set = np.load('data/noise_y_set.npy')

In [5]:
use_obstacle = args.use_obstacle #false for standard, true for gianni's formulation
use_noise = args.use_noise
MPC_HORIZON = args.MPC_HORIZON
if use_obstacle:
    if use_noise is False:
        PROJECT_PATH = DATA_ROOT + 'gianni_new/' 
        noise_y_set *= 0.
    else:
        PROJECT_PATH = DATA_ROOT + 'gianni_noise/' 
else:
    if use_noise is False:
        PROJECT_PATH = DATA_ROOT + 'standard/'
        noise_y_set *= 0.
    else:
        PROJECT_PATH = DATA_ROOT + 'standard_noise/' 

#env_name = '../env/urdf/full_stair_small.urdf'
np.set_printoptions(precision=3, suppress=True)

#### Load contact sequence data

#### Determine the environment
if args.env_name == 'more_closer_full_stair_small_0.15':
    total_data = np.load('data/more_closer_full_stair_small_0.15.npy', allow_pickle=True)[()]
elif args.env_name == 'full_stair_small_0.15':
    total_data = np.load('data/2full_stair_small_0.15.npy', allow_pickle=True)[()]
elif args.env_name == 'full_stair_small_0.2':
    total_data = np.load('data/2full_stair_small_0.2.npy', allow_pickle=True)[()]
elif args.env_name == 'full_stair_small':
    total_data = np.load('data/2full_stair_small.npy', allow_pickle=True)[()]
elif args.env_name == 'full_stair':
    total_data = np.load('data/2full_stair.npy', allow_pickle=True)[()]
elif args.env_name == 'full_stair_small_slippery':
    total_data = np.load('data/2full_stair_small.npy', allow_pickle=True)[()]
else:
    print('Env name {} does not exist'.format(args.env_name))
    raise Exception
    
env_name = total_data[0]['env_name']
if args.env_name == 'full_stair_small_slippery':
    env_name = '../env/urdf/full_stair_small_slippery.urdf'
    
stairHeight = total_data[0]['stairHeight']
N_STEP = total_data[0]['N_STEP']

PROJECT_PATH += env_name[12:-5] + '/'

## Pybullet Visualizations

#### Setup Pybullet
if args.use_gui:
    p.connect(p.GUI)
else:
    p.connect(p.DIRECT)
    
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

robot_id, plane_id, env_id = setup_pybullet(urdf_name, env_name)

pb_joint_indices = np.array([1, 2, 3, 11, 12, 13, 6, 7, 8, 16, 17, 18])
set_q_std = partial(set_q, robot_id = robot_id, joint_indices=pb_joint_indices, set_base = True)
vis_traj_std = partial(vis_traj, vis_func=set_q_std, dt = 0.001)

#### Load pinocchio robot

rmodel, rdata, q0, v0, rh_id, rf_id, lh_id, lf_id = load_robot_pinocchio()

### Define the obstacle (from the scene)

scene = generate_stair_small(stepHeight=stairHeight, side_margin=0.1)
obstacles = create_obstacle_from_scene(scene, stair_height=stairHeight, obs_weight=args.obs_weight, use_full_size=True) 
###***** actual_radius to be checked. 


# obstacles[1]['radius'] = stairHeight/2 + 0.05
# obstacles[1]['weight'] = 500

#### Define MPC parameters

# Create the MPC application
params_caracal = caracal.CaracalParams()
params = caracal.CaracalParams(mpc_filename)
params.solverVerbose = args.solver_verbose
params.withForceReg = True
params.withImpulseReg = True
#teguh
params.start_obs_time = 1
params.end_obs_time = args.N_ss-1
params.Qx_JointVel = 1.4
params.w_footcontact = 1
params.w_footcontactvel = 1

stepHeight = stairHeight

N_ss = args.N_ss
N_ds = args.N_ds
ws = np.ones((N_ss,2))
ws[:,0] *= 1e6
ws[:,1] *= 1e4
if use_obstacle:
    ws[2:-2] *= 0

## Batch Computation

/idiap/temp/tlembono/miniconda3/envs/robotpkg/lib/python3.8/site-packages/pinocchio/shortcuts.py:45: UserWarning: You passed package dir(s) via argument geometry_model and provided package_dirs.
  geom_model = pin.buildGeomFromUrdf(model, filename, geometry_type, package_dirs)


In [6]:
import copy
def compute_sim(data, n_iter, use_obstacle = False, use_noise = False, noise_y = 20., noise_t = 2000, MPC_HORIZON = 60):
    '''
    Computing one full simulation of a trotting gait on stairs
    '''
    q0[:3] = data['q_init']
    cs = create_contact_sequence_slim_to_caracal(data['rh_poses'], data['rf_poses'], data['lh_poses'], data['lf_poses'])
    gait = create_gait_trot(cs, N_ds = N_ds, N_ss = N_ss, stepHeight=stepHeight, ws = ws)
    N_MPC_STEPS = gait.T
    #### Initialize MPC
    robot_id, plane_id, env_id = setup_pybullet(urdf_name, env_name)
    set_q_std(q0)

    params.solverIterations = n_iter
    if use_obstacle:
        mpc = caracal.Caracal(q0, rmodel, gait, MPC_HORIZON, params, obstacles)
    else:
        mpc = caracal.Caracal(q0, rmodel, gait, MPC_HORIZON, params)
        
    mpc.start(q0, maxiter=100)
    simulator = Simulator(mpc, plane_id=plane_id, robot_id=robot_id, rmodel=rmodel)

    #### Run MPC
    xs_opt,us_opt,ffeas,xs_ref,us_ref  = [],[],[],[],[]
    status_finish = True
    try:
        for i in range(N_MPC_STEPS*10):
            if use_noise and (noise_t < i < noise_t+1000):
                p.applyExternalForce(robot_id, -1, [0, noise_y,0], [0,0,0], p.LINK_FRAME)
            x, u, feas_i, x_ref, u_ref = simulator.step()
            if np.linalg.norm(x[:3]) > 5:
                raise Exception
            if np.isnan(x).any(): 
                raise Exception
            if simulator.i == 1:
                xs_opt.append(copy.deepcopy(x))
                us_opt.append(copy.deepcopy(u))
                ffeas.append(copy.deepcopy(feas_i))
                xs_ref.append(copy.deepcopy(x_ref))
                us_ref.append(copy.deepcopy(u_ref))
    except:
        status_finish = False
        
    print('Finish successfully')
    coms = np.array([pinocchio.centerOfMass(rmodel, rdata, x[:19]) for x in xs_opt])
    foot_poses = np.array([compute_foot_pose(rmodel, rdata, x[:19]) for x in xs_opt])
    target_pose = compute_foot_pose(rmodel, rdata, xs_opt[-1][:19])

    res = dict()
    res['xs'] = np.array(xs_opt)
    res['us'] = np.array(us_opt)
    res['xs_ref'] = np.array(xs_ref)
    res['us_ref'] = np.array(us_ref)
    res['ffeas'] = np.array(ffeas)
    res['coms'] = coms
    res['cs'] = cs
    res['N_ds'] = N_ds
    res['N_ss'] = N_ss
    res['stepHeight'] = stepHeight
    res['ws'] = ws

    res['foot_poses'] = foot_poses
    res['target_pose'] = target_pose
    res['params'] = params
    res['n_iter'] = mpc._params.solverIterations
    res['status_finish'] = status_finish
    res['obs_weight'] = args.obs_weight
    if use_noise:
        res['noise_t'] = noise_t
        res['noise_y'] = noise_y
        
    return res

tic = time.time()

res_data = []
fail_set = []

In [7]:
data = total_data[0]
n_iter = 1

In [8]:
q0[:3] = data['q_init']
cs = create_contact_sequence_slim_to_caracal(data['rh_poses'], data['rf_poses'], data['lh_poses'], data['lf_poses'])
gait = create_gait_trot(cs, N_ds = N_ds, N_ss = N_ss, stepHeight=stepHeight, ws = ws)
N_MPC_STEPS = gait.T
#### Initialize MPC
robot_id, plane_id, env_id = setup_pybullet(urdf_name, env_name)
set_q_std(q0)

params.solverIterations = n_iter
if use_obstacle:
    mpc = caracal.Caracal(q0, rmodel, gait, MPC_HORIZON, params, obstacles)
else:
    mpc = caracal.Caracal(q0, rmodel, gait, MPC_HORIZON, params)

mpc.start(q0, maxiter=100)


/idiap/temp/tlembono/openrobots/lib/python3.8/site-packages/caracal/mpc.py:568: UserWarning: Deprecated. Use property active_set
  contacts = model.differential.contacts.active
/idiap/temp/tlembono/openrobots/lib/python3.8/site-packages/caracal/mpc.py:569: UserWarning: Deprecated. Use property active_set
  nc = len(model.differential.contacts.active)


In [14]:
from gridtk.tools import get_array_job_slice
a = np.arange(len(total_data))
a = a[get_array_job_slice(len(a))]
toc = 1.
for i in range(1):#a:
    for n_iter in [1]:#:, 2, 5]:
        #robot_id, plane_id, env_id = setup_pybullet(urdf_name, env_name)
        noise_y, noise_t = noise_y_set[i], noise_t_set[i]
        
        print('Planning {} with {} iterations'.format(i, n_iter))
        toc = time.time()
        print('Planning so far takes {} seconds'.format(toc-tic))       
        data = total_data[i]
        try:
            res = compute_sim(data, n_iter, use_obstacle=use_obstacle, use_noise = use_noise, noise_y = noise_y, noise_t = noise_t, MPC_HORIZON=MPC_HORIZON)
        except BaseException as err:
            fail_set += ['data_{}_{}_iter'.format(i,n_iter)]
            print(err)
            continue
        res_data.append(res)
        
        if res['status_finish'] is False or check_traj(res['xs']) is False:
            cur_file_name = PROJECT_PATH + '{}_iter_{}_horizon_fail/res_{}.npy'.format(n_iter, MPC_HORIZON, i)
            cur_dir = PROJECT_PATH + '{}_iter_{}_horizon_fail'.format(n_iter, MPC_HORIZON,  i)
        else:
            cur_file_name = PROJECT_PATH + '{}_iter_{}_horizon/res_{}.npy'.format(n_iter, MPC_HORIZON, i)
            cur_dir = PROJECT_PATH + '{}_iter_{}_horizon'.format(n_iter, MPC_HORIZON, i)
        if os.path.exists(cur_dir) is False:
            os.makedirs(cur_dir)

        np.savez_compressed(cur_dir +'/res_{}'.format(i), res=res)

print('Planning takes {} seconds'.format(toc-tic))

np.save(PROJECT_PATH + 'failset.npy', fail_set)

Planning 0 with 1 iterations
Planning so far takes 92.79789352416992 seconds


/idiap/temp/tlembono/openrobots/lib/python3.8/site-packages/caracal/mpc.py:568: UserWarning: Deprecated. Use property active_set
  contacts = model.differential.contacts.active
/idiap/temp/tlembono/openrobots/lib/python3.8/site-packages/caracal/mpc.py:569: UserWarning: Deprecated. Use property active_set
  nc = len(model.differential.contacts.active)
/idiap/temp/tlembono/openrobots/lib/python3.8/site-packages/caracal/mpc.py:560: UserWarning: Deprecated. Use property active_set
  contacts = model.impulses.active
/idiap/temp/tlembono/openrobots/lib/python3.8/site-packages/caracal/mpc.py:561: UserWarning: Deprecated. Use property active_set
  nc = len(model.impulses.active)


Finish successfully


/tmp/ipykernel_184330/232917850.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res['us_ref'] = np.array(us_ref)


Planning takes 92.79789352416992 seconds
